
<font size="20"><h3 align='center'>**_DSL Project_**</font><br>
    
    
<font size="5"><h3 align='center'><span style='color:Blue'> Predict impact of air quality on mortality rates(Kaggle Competition)</font>


<font size="3"><h3 align='center'>https://www.kaggle.com/c/predict-impact-of-air-quality-on-death-rates/overview</font><br>
    <br>


<font size="3">**Group Number - 2**<br>
<font size="3"> Mimansi Agarwal   &emsp;19BM6JP15</font>        
<font size="3"> Vaibhav HK Anand  &emsp;19BM6JP24</font>    
<font size="3"> Bankar Prasad Vilas &emsp;19BM6JP25</font> 


# Library

In [2]:

import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import xgboost as xgb
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns
from impyute.imputation.cs import mice
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import  make_scorer
import math

# Data import

In [5]:
train = pd.read_csv('full_data_imputation_train2.csv', parse_dates=['date'], index_col='Id')
train_region = pd.read_csv('region_wise_imputed_train.csv', parse_dates=['date'], index_col='Id')
test = pd.read_csv('test.csv', parse_dates=['date'], index_col='Id')
region=pd.read_csv('regions.csv')

In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18403 entries, 1 to 16425
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   region          18403 non-null  object        
 1   date            18403 non-null  datetime64[ns]
 2   mortality_rate  18403 non-null  float64       
 3   O3              18403 non-null  float64       
 4   PM10            18403 non-null  float64       
 5   PM25            18403 non-null  float64       
 6   NO2             18403 non-null  float64       
 7   T2M             18403 non-null  float64       
 8   Code            18403 non-null  object        
 9   Region          18403 non-null  object        
dtypes: datetime64[ns](1), float64(6), object(3)
memory usage: 1.5+ MB


**_Missing values present in 4 columns, O3 ,PM10, PM25, NO2_**

In [7]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7886 entries, 18404 to 26289
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   region  7886 non-null   object        
 1   date    7886 non-null   datetime64[ns]
 2   O3      7886 non-null   float64       
 3   PM10    7886 non-null   float64       
 4   PM25    7886 non-null   float64       
 5   NO2     7886 non-null   float64       
 6   T2M     7886 non-null   float64       
dtypes: datetime64[ns](1), float64(5), object(1)
memory usage: 492.9+ KB


In [8]:
region

,Code,Region
0,E12000001,North East
1,E12000002,North West
2,E12000003,Yorkshire and The Humber
3,E12000004,East Midlands
4,E12000005,West Midlands
5,E12000006,East of England
6,E12000007,London
7,E12000008,South East
8,E12000009,South West


In [9]:
region.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Code    9 non-null      object
 1   Region  9 non-null      object
dtypes: object(2)
memory usage: 272.0+ bytes


# Exploratory data analysis

In [10]:
region.head()

,Code,Region
0,E12000001,North East
1,E12000002,North West
2,E12000003,Yorkshire and The Humber
3,E12000004,East Midlands
4,E12000005,West Midlands


#  Imputation

## Impution on full data

#  Feature Engeering

one hot encoding for regions

In [11]:
dummy1= pd.get_dummies(train['region'])
train_f = train.join(dummy1)

dummy2= pd.get_dummies(test['region'])
test_f = test.join(dummy2)

In [12]:
train_f.shape,test_f.shape

((18403, 19), (7886, 16))

doy and year

In [13]:
train_f['year'] = train_f['date'].map(lambda x: x.year)
test_f['year'] = test_f['date'].map(lambda x: x.year)

train_f['year^2'] = train_f['year']**2
test_f['year^2'] = test_f['year']**2

train_f['doy'] = train_f['date'].map(lambda x: x.dayofyear)
test_f['doy'] = test_f['date'].map(lambda x: x.dayofyear)

train_f['doy^2'] = train_f['doy']**2
test_f['doy^2'] = test_f['doy']**2

Log Transformation

In [14]:
train_f['NO2_log']=np.log(train_f['NO2'])
train_f['PM10_log']=np.log(train_f['PM10'])
train_f['PM25_log']=np.log(train_f['PM25'])

test_f['NO2_log']=np.log(test_f['NO2'])
test_f['PM10_log']=np.log(test_f['PM10'])
test_f['PM25_log']=np.log(test_f['PM25'])

C:\Users\miman\Anaconda3\lib\site-packages\pandas\core\series.py:679: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [15]:
train_f.info(),test_f.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18403 entries, 1 to 16425
Data columns (total 26 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   region          18403 non-null  object        
 1   date            18403 non-null  datetime64[ns]
 2   mortality_rate  18403 non-null  float64       
 3   O3              18403 non-null  float64       
 4   PM10            18403 non-null  float64       
 5   PM25            18403 non-null  float64       
 6   NO2             18403 non-null  float64       
 7   T2M             18403 non-null  float64       
 8   Code            18403 non-null  object        
 9   Region          18403 non-null  object        
 10  E12000001       18403 non-null  uint8         
 11  E12000002       18403 non-null  uint8         
 12  E12000003       18403 non-null  uint8         
 13  E12000004       18403 non-null  uint8         
 14  E12000005       18403 non-null  uint8         
 15  E1

(None, None)

In [16]:
total_f=total=pd.concat([train_f,test_f],sort=False)

In [17]:
total_f['T2M_rolling'] = total_f.groupby('region')['T2M'].transform(lambda x: x.rolling(30).mean())

In [18]:
total_f.shape

(26289, 27)

In [19]:
train_f=total_f[:18403].dropna()
test_f=total_f[18403:].drop(['mortality_rate'],axis=1)

In [20]:
train_f.shape,test_f.shape

((18026, 27), (7886, 26))

#  Modelling - Regression based models

In [21]:
X_train, X_test, y_train, y_test = train_test_split(train_f.drop(['mortality_rate'],axis=1), train_f['mortality_rate'], test_size=0.2)

In [22]:
X_train.columns

Index(['region', 'date', 'O3', 'PM10', 'PM25', 'NO2', 'T2M', 'Code', 'Region',
       'E12000001', 'E12000002', 'E12000003', 'E12000004', 'E12000005',
       'E12000006', 'E12000007', 'E12000008', 'E12000009', 'year', 'year^2',
       'doy', 'doy^2', 'NO2_log', 'PM10_log', 'PM25_log', 'T2M_rolling'],
      dtype='object')

In [23]:
RMSE= make_scorer(metrics.mean_squared_error)

In [24]:
predictors=['O3','PM10','PM25','NO2','T2M']

In [25]:
X=train_f.drop(['mortality_rate'],axis=1)
y=train_f['mortality_rate']

In [26]:
models = [
    LinearRegression(),
    linear_model.Lasso(alpha=0.1),
    Ridge(alpha=1.0),
    RandomForestRegressor(max_depth=2, random_state=0),
]
CV = 5
for model in models:
    model_name = model.__class__.__name__
    
    accuracies = cross_val_score(model,X[predictors], y,scoring=RMSE, cv=CV)
    
    print(model_name,"RMSE",np.sqrt(accuracies),"\n")
    print("average RMSE :", np.sqrt(accuracies).mean(),"\n","\n")

LinearRegression RMSE [0.27740824 0.22263904 0.2296782  0.29398905 0.22082776] 

average RMSE : 0.2489084590679548 
 

Lasso RMSE [0.29014054 0.22482052 0.22348427 0.35580941 0.22816987] 

average RMSE : 0.26448492220734127 
 

Ridge RMSE [0.27740839 0.22263898 0.22967804 0.29398968 0.2208278 ] 

average RMSE : 0.24890857691268659 
 

RandomForestRegressor RMSE [0.31499045 0.22675202 0.22852175 0.36930784 0.24331041] 

average RMSE : 0.27657649393580575 
 



In [27]:
predictors=['O3','PM10','PM25','NO2_log','T2M']

In [28]:
models = [
    LinearRegression(),
    linear_model.Lasso(alpha=0.1),
    Ridge(alpha=1.0),
    RandomForestRegressor(max_depth=2, random_state=0),
]
CV = 5
for model in models:
    model_name = model.__class__.__name__
    
    accuracies = cross_val_score(model,X[predictors], y,scoring=RMSE, cv=CV)
    
    print(model_name,"RMSE",np.sqrt(accuracies),"\n")
    print("average RMSE :", np.sqrt(accuracies).mean(),"\n","\n")

LinearRegression RMSE [0.2830161  0.2321538  0.22291807 0.33393921 0.22972073] 

average RMSE : 0.2603495841144592 
 

Lasso RMSE [0.31439311 0.22857669 0.21907402 0.36463865 0.23717917] 

average RMSE : 0.27277233060924233 
 

Ridge RMSE [0.28301772 0.23214866 0.2229133  0.33395014 0.2297185 ] 

average RMSE : 0.26034966504704476 
 

RandomForestRegressor RMSE [0.31499045 0.22675202 0.22852175 0.36930784 0.24331041] 

average RMSE : 0.27657649393580575 
 



In [29]:
predictors=['O3','PM10','PM25_log','NO2_log','T2M']

In [30]:
models = [
    LinearRegression(),
    linear_model.Lasso(alpha=0.1),
    Ridge(alpha=1.0),
    RandomForestRegressor(max_depth=2, random_state=0),
]
CV = 5
for model in models:
    model_name = model.__class__.__name__
    
    accuracies = cross_val_score(model,X[predictors], y,scoring=RMSE, cv=CV)
    
    print(model_name,"RMSE",np.sqrt(accuracies),"\n")
    print("average RMSE :", np.sqrt(accuracies).mean(),"\n","\n")

LinearRegression RMSE [0.28339697 0.23197316 0.22308032 0.33268263 0.22988701] 

average RMSE : 0.26020401718740854 
 

Lasso RMSE [0.31461765 0.2285767  0.21907402 0.36463865 0.23717917] 

average RMSE : 0.27281724004277214 
 

Ridge RMSE [0.28339655 0.23196874 0.22307519 0.33269324 0.22988381] 

average RMSE : 0.26020350685584703 
 

RandomForestRegressor RMSE [0.31499045 0.22675202 0.22852175 0.36930784 0.24331041] 

average RMSE : 0.27657649393580575 
 



In [31]:
predictors=['O3','PM10_log','PM25_log','NO2_log','T2M']

In [32]:
models = [
    LinearRegression(),
    linear_model.Lasso(alpha=0.1),
    Ridge(alpha=1.0),
    RandomForestRegressor(max_depth=2, random_state=0),
]
CV = 5
for model in models:
    model_name = model.__class__.__name__
    
    accuracies = cross_val_score(model,X[predictors], y,scoring=RMSE, cv=CV)
    
    print(model_name,"RMSE",np.sqrt(accuracies),"\n")
    print("average RMSE :", np.sqrt(accuracies).mean(),"\n","\n")

LinearRegression RMSE [0.28380724 0.23174178 0.22251717 0.33282088 0.22922977] 

average RMSE : 0.26002336744933696 
 

Lasso RMSE [0.31568593 0.2277781  0.21726415 0.36463865 0.23539886] 

average RMSE : 0.27215313881213865 
 

Ridge RMSE [0.28380729 0.23173404 0.22251363 0.33282413 0.22922972] 

average RMSE : 0.26002176092700985 
 

RandomForestRegressor RMSE [0.31499045 0.22675202 0.22852175 0.36930784 0.24331041] 

average RMSE : 0.27657649393580575 
 



In [33]:
predictors=['O3','PM10_log','PM25_log','NO2_log','T2M_rolling']

In [34]:
models = [
    LinearRegression(),
    linear_model.Lasso(alpha=0.1),
    Ridge(alpha=1.0),
    RandomForestRegressor(max_depth=2, random_state=0),
]
CV = 5
for model in models:
    model_name = model.__class__.__name__
    
    accuracies = cross_val_score(model,X[predictors], y,scoring=RMSE, cv=CV)
    
    print(model_name,"RMSE",np.sqrt(accuracies),"\n")
    print("average RMSE :", np.sqrt(accuracies).mean(),"\n","\n")

LinearRegression RMSE [0.26433114 0.20830041 0.20349179 0.32079179 0.21864372] 

average RMSE : 0.24311176902847764 
 

Lasso RMSE [0.30325015 0.2099866  0.20041842 0.36398766 0.22502729] 

average RMSE : 0.2605340270236548 
 

Ridge RMSE [0.26432754 0.20829512 0.2034868  0.32079834 0.21864108] 

average RMSE : 0.24310977533949368 
 

RandomForestRegressor RMSE [0.29967446 0.20992581 0.20549142 0.35969427 0.23310896] 

average RMSE : 0.26157898288070847 
 



In [35]:
predictors=['O3','PM10_log','PM25_log','NO2_log','T2M_rolling','year','year^2','doy','doy^2']

In [36]:
models = [
    LinearRegression(),
    linear_model.Lasso(alpha=0.1),
    Ridge(alpha=1.0),
    RandomForestRegressor(max_depth=2, random_state=0),
]
CV = 5
for model in models:
    model_name = model.__class__.__name__
    
    accuracies = cross_val_score(model,X[predictors], y,scoring=RMSE, cv=CV)
    
    print(model_name,"RMSE",np.sqrt(accuracies),"\n")
    print("average RMSE :", np.sqrt(accuracies).mean(),"\n","\n")

LinearRegression RMSE [0.25163089 0.20103549 0.18888543 0.31984291 0.20042408] 

average RMSE : 0.23236376032335054 
 

Lasso RMSE [0.29774598 0.20724945 0.18265143 0.37033586 0.19596881] 

average RMSE : 0.25079030592275003 
 

Ridge RMSE [0.25159018 0.2009512  0.18891731 0.31993199 0.2001012 ] 

average RMSE : 0.23229837688412766 
 

RandomForestRegressor RMSE [0.30250254 0.20900642 0.20463965 0.36274549 0.23063014] 

average RMSE : 0.261904849424171 
 



In [37]:
predictors=['T2M_rolling', 'year', 'year^2', 'doy', 'doy^2','E12000001', 'E12000002', 'E12000003', 'E12000004','E12000005', 'E12000006', 'E12000007', 'E12000008']

In [38]:
models = [
    LinearRegression(),
    linear_model.Lasso(alpha=0.1),
    Ridge(alpha=1.0),
    RandomForestRegressor(max_depth=2, random_state=0),
]
CV = 5
for model in models:
    model_name = model.__class__.__name__
    
    accuracies = cross_val_score(model,X[predictors], y,scoring=RMSE, cv=CV)
    
    print(model_name,"RMSE",np.sqrt(accuracies),"\n")
    print("average RMSE :", np.sqrt(accuracies).mean(),"\n","\n")

LinearRegression RMSE [0.22857539 0.18480372 0.19915    0.16379925 0.19331738] 

average RMSE : 0.1939291499730676 
 

Lasso RMSE [0.30253048 0.20273216 0.17838908 0.37580078 0.19765775] 

average RMSE : 0.2514220509883692 
 

Ridge RMSE [0.22869481 0.18463661 0.19900196 0.15918251 0.19291681] 

average RMSE : 0.192886539077443 
 

RandomForestRegressor RMSE [0.29391025 0.21333037 0.23101697 0.36274549 0.22699476] 

average RMSE : 0.26559956676487956 
 



In [39]:
predictors1=['NO2_log', 'year', 'year^2', 'doy', 'doy^2','PM10_log']
predictors2=['T2M_rolling', 'year', 'year^2', 'doy', 'doy^2','E12000001', 'E12000002', 'E12000003', 'E12000004','E12000005', 'E12000006', 'E12000007', 'E12000008']
predictors3=['E12000001', 'E12000002', 'E12000003', 'E12000004','E12000005', 'E12000006', 'E12000007', 'E12000008']

In [40]:
pred = [
    predictors1,
    predictors2,
    predictors3,
]
for preds in pred:
    #model_name = model.__class__.__name__
    
    accuracies = cross_val_score(LinearRegression(),X[preds], y,scoring=RMSE, cv=CV)
    
    print(model_name,"RMSE",np.sqrt(accuracies),"\n")
    print("average RMSE :", np.sqrt(accuracies).mean(),"\n","\n")

RandomForestRegressor RMSE [0.27435974 0.21754495 0.19883702 0.34278609 0.19881749] 

average RMSE : 0.24646905902503238 
 

RandomForestRegressor RMSE [0.22857539 0.18480372 0.19915    0.16379925 0.19331738] 

average RMSE : 0.1939291499730676 
 

RandomForestRegressor RMSE [3.20813764e-01 2.75481076e-01 2.81456941e-01 2.46364649e-01
 4.94802667e+10] 

average RMSE : 9896053349.415403 
 



## Final Algo - mixed LR

In [41]:
a = LinearRegression()
b = LinearRegression()
c = LinearRegression()
ab = LinearRegression()

In [42]:

y0 = train_f['mortality_rate']



#modela
a_predictors = ['NO2_log', 'year', 'year^2', 'doy', 'doy^2','PM10_log']
a_X0 = train_f[a_predictors]
a.fit(a_X0, y0)



#modelb
b_predictors = [
        'T2M_rolling', 'year', 'year^2', 'doy', 'doy^2',
        'E12000001', 'E12000002', 'E12000003', 'E12000004',
        'E12000005', 'E12000006', 'E12000007', 'E12000008'
        ]
b_X0 = train_f[b_predictors]
b.fit(b_X0, y0)



#modelc
c_predictors = [
        'E12000001', 'E12000002', 'E12000003', 'E12000004',
        'E12000005', 'E12000006', 'E12000007', 'E12000008'
        ]
c_X0 = train_f[c_predictors]
c.fit(c_X0, y0)


region_effect_reduction = 0.85


# Model AB
# Produces the final solution. Combines A and "reduced B" by using their predictions as features for a linear regression.

ab_X0 = pd.DataFrame({
    'A': a.predict(a_X0),
    'reduced B': b.predict(b_X0) - (c.predict(c_X0) * region_effect_reduction)
    })

ab.fit(ab_X0, y0)

# predict

a_X1 = test_f[a_predictors]
b_X1 = test_f[b_predictors]
c_X1 = test_f[c_predictors]

ab_X1 = pd.DataFrame({'A': a.predict(a_X1),'reduced B': b.predict(b_X1) - (c.predict(c_X1) * region_effect_reduction)})

ab_y1 = ab.predict(ab_X1)



predictions = pd.DataFrame(index=test_f.index)
predictions['mortality_rate'] =  a.predict(a_X1)
predictions.to_csv('algo_final_train_full1.csv')


# Results
best LB socre for -(region_effect_reduction = 0.85)<br>
private score - 0.28948<br>
public score - 0.28784<br>


benchmark -<br>
private score - 0.29022<br>
public socre - 0.29259<br>